In [21]:
import pandas as pd
import numpy as np
import datetime
import random
from itertools import cycle
from keras.models import Sequential
from keras.layers import Dense, LSTM
from math import sqrt, floor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

## Load Data

In [38]:
df = pd.read_csv("tsy_data_2019_q3.csv")

df.head(3)

,Trade date,Settlement date,Transaction category,"Trade amount (in millions, USD)",Issuer,Security description,CUSIP,Price,"Accrued interest (in dollars, USD)","Total amount transferred (in millions, USD)",Counterparty,Unnamed: 11,Unnamed: 12
0,Jul 25 2019,Jul 26 2019,Purchase,10.0,TSY,TFR 00.115 01/31/21,9128285Y2,99.900078,"57,368.74",10.0,Amherst Pierpont Securities LLC,NaN,NaN
1,Jul 25 2019,Jul 26 2019,Purchase,3.0,TSY,TFR 00.033 04/30/20,9128284K3,99.902500,"16,616.12",3.0,Cantor Fitzgerald & Co.,NaN,NaN
2,Jul 25 2019,Jul 26 2019,Purchase,10.0,TSY,TFR 00.033 04/30/20,9128284K3,99.871875,"55,387.07",10.0,"Credit Suisse AG, New York Branch",NaN,NaN


In [37]:
tradeDates = pd.unique(df['Trade date'])
dataDict = {}
for x in tradeDates:
    if x not in dataDict.keys():
        dataDict[x] = 0
    for amount in df[(df['Trade date'] == x)]['Trade amount (in millions, USD)']:
        dataDict[x] += float(amount.replace(',',''))

print(dataDict)
newDf = pd.DataFrame(data={'Trade date': dataDict.keys(), 'Trade Amount': dataDict.values()})
newDf

{'Jul 25 2019': 50.0, 'Aug 14 2019': 3001.0, 'Aug 15 2019': 1601.0, 'Aug 19 2019': 1801.0, 'Aug 20 2019': 1801.0, 'Aug 22 2019': 1601.0, 'Aug 26 2019': 2201.0, 'Aug 27 2019': 2401.0, 'Sep 04 2019': 1401.0, 'Sep 06 2019': 2199.0, 'Sep 09 2019': 1601.0, 'Sep 10 2019': 401.0, 'Sep 16 2019': 1601.0, 'Sep 17 2019': 3001.0, 'Sep 19 2019': 1801.0, 'Sep 25 2019': 1801.0, 'Sep 26 2019': 1601.0, 'Sep 27 2019': 2201.0, 'Sep 30 2019': 2401.0}


,Trade date,Trade Amount
0,Jul 25 2019,50.0
1,Aug 14 2019,3001.0
2,Aug 15 2019,1601.0
3,Aug 19 2019,1801.0
4,Aug 20 2019,1801.0
5,Aug 22 2019,1601.0
6,Aug 26 2019,2201.0
7,Aug 27 2019,2401.0
8,Sep 04 2019,1401.0
9,Sep 06 2019,2199.0
